In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import cPickle as pickle
import scipy.special
import ghalton
import numpy as np
import time
import os
import sys
sys.path.insert(1,'/Users/zyzdiana/GitHub/AC297r-Volume-Registration/code')

In [3]:
# load data
data_dict = pickle.load(open('data_dict.p','rb'))

In [4]:
# define golden ratio
gr = (np.sqrt(5)-1)/2
def golden_section_search(func, a, b , tol=1e-5, *args):
    t0 = time.time()
    '''
    Finds the minimum of function on [a,b]
    tol: specifies stopping criteria
    *args: handles the rest of the input arguments for func
    '''
    # calculate a new interval from golden ratio
    c = b - gr * (b-a)
    d = a + gr * (b-a)
    # loop until finding the minimum
    while abs(c-d) > tol:
        print c,d
        fc = func(c,*args)
        fd = func(d,*args)
        if fc < fd:
            b = d
            d = c  #fd=fc;fc=f(c)
            c = b - gr*(b-a)
        else:
            a = c
            c = d  #fc=fd;fd=f(d)
            d = a + gr*(b-a)
    print 'time: ', time.time()-t0
    return (b+a)/2

In [5]:
# Bessel Rotation
def to_radian(theta):
    return theta*np.pi/180.

def circle_mask(image):
    ox = image.shape[1]/2.-0.5
    oy = image.shape[0]/2.-0.5
    r = image.shape[0]/2.-0.5
    y, x = np.ogrid[-ox:image.shape[0]-ox, -oy:image.shape[0]-oy]
    mask = x*x + y*y <= r*r
    image[~mask] = 0
    return image

def cf_ssd(J, I):
    return np.sum((J-I)**2)

def bessel_rotate(image, theta, mask = False, r = 8):
    Ib = np.zeros(image.shape)
    theta = to_radian(theta)
    s = (image.shape[0]-1)/2.

    x = np.linspace(-s, s, image.shape[1])
    y = np.linspace(-s, s, image.shape[0])
    
    xx, yy = np.meshgrid(x,y)
    
    rM = np.array([[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]])

    for i in np.arange(-s,s+1):
        for j in np.arange(-s,s+1):
            x = np.dot(rM, np.array([i,j]))

            if(np.sum(abs(x)>s)):
                Ib[i+s,j+s]=0
                
            else:
                R = np.sqrt((xx-x[1])**2 + (yy-x[0])**2)
                mask_R = (R == 0)
                mask_R_le = (R <= r) & (R > 0)
                Bess = np.zeros(R.shape)
                Bess[mask_R_le] = scipy.special.j1(np.pi*R[mask_R_le])/(np.pi*R[mask_R_le]) #*np.hanning(R)
                Bess[mask_R] = 0.5
                tmp = image*Bess
                Ib[i+s,j+s] = np.sum(tmp)*np.pi/2
    if(mask):
        Ib = circle_mask(Ib)
    return Ib
  

def bessel_rotate_halton(image, theta, x1, y1):
    Ib = []
    theta = to_radian(theta)
    s = (image.shape[0]-1)/2.
    
    rM = np.array([[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]])
    x = []
    for i in np.arange(-s,s+1):
        for j in np.arange(-s,s+1):
            x.append(np.dot(rM, np.array([i,j])))
    x = np.array(x)
    for idx in xrange(len(x1)):
        R = np.sqrt((x[:,1]-x1[idx])**2 + (x[:,0]-y1[idx])**2)
        mask_R = (R == 0)
        Bess = np.zeros(R.shape)
        Bess[~mask_R] = scipy.special.j1(np.pi*R[~mask_R])/(np.pi*R[~mask_R])
        Bess[mask_R] = 0.5
        #Bess = Bess/(2*np.pi*np.sum(Bess*R))
        tmp = image.ravel()*Bess
        Ib.append(np.sum(tmp)*np.pi/2)
    return np.array(Ib)

def bessel_halton_cost_func(vol1, vol2, N, thetas, axis):
    '''
    vol1: original image
    vol2: volume to be rotated
    thetas: list of degress to try
    cf: cost function
    arg: string for plot titles
    '''
    cost_func = np.zeros([len(thetas),])
    # generate Halton sample points
    s = (len(vol1)-1)/2.
    sequencer = ghalton.GeneralizedHalton(ghalton.EA_PERMS[:3])
    sequencer.reset()
    points = sequencer.get(N)
    pts = np.array(points)
    x1 = (len(vol1)-1) * pts[:,0] - s
    y1 = (len(vol1)-1) * pts[:,1] - s
    new_vol1 = np.zeros([len(vol1),N])
    for i in xrange(len(vol1)):
        if(axis == 0):
            sub1 = circle_mask(vol1[i,:,:])
        elif(axis == 1):
            sub1 = circle_mask(vol1[:,i,:])
        else:
            sub1 = circle_mask(vol1[:,:,i])
        rot = bessel_rotate_halton(sub1, 0, x1, y1)
        new_vol1[i] = rot
    for idx, t in enumerate(thetas):
        print t, 
        new_vol2 = np.empty([len(vol2),N])
        for i in xrange(len(vol2)):
            if(axis==0):
                sub2 = circle_mask(vol2[i,:,:])
            elif(axis==1):
                sub2 = circle_mask(vol2[:,i,:])
            else:
                sub2 = circle_mask(vol2[:,:,i])
            rot = bessel_rotate_halton(sub2, t, x1, y1)
            new_vol2[i] = rot
        cost_func[idx] = cf_ssd(new_vol2,new_vol1)
    return cost_func

def bessel_cost(theta, vol1, vol2, axis, mask=True):
    '''
    vol1: original image
    vol2: volume to be rotated
    theta: angle of rotation
    axis: axis of rotation
    cf: cost function
    '''

    new_vol2 = np.ones(vol2.shape)
    for i in xrange(len(vol2)):
        if(axis == 0):
            sub = vol2[i,:,:]
            if(mask):
                vol1[i,:,:] = circle_mask(vol1[i,:,:])
        elif(axis == 1):
            sub = vol2[:,i,:]
            if(mask):
                vol1[:,i,:] = circle_mask(vol1[:,i,:])
        else:
            sub = vol2[:,:,i]
            if(mask):
                vol1[:,:,i] = circle_mask(vol1[:,:,i])

        rot = bessel_rotate(sub, theta, mask)

        if(axis == 0):
            new_vol2[i,:,:] = rot
        elif(axis == 1):
            new_vol2[:,i,:] = rot
        else:
            new_vol2[:,:,i] = rot
            
    return cf_ssd(new_vol2,vol1)

def bessel_halton_cost_func_circle(vol1, vol2, N, thetas, axis):
    '''
    vol1: original image
    vol2: volume to be rotated
    thetas: list of degress to try
    cf: cost function
    arg: string for plot titles
    '''
    cost_func = np.zeros([len(thetas),])
    # generate Halton sample points
    s = (len(vol1)-1)/2.
    sequencer = ghalton.GeneralizedHalton(ghalton.EA_PERMS[:3])
    sequencer.reset()
    points = sequencer.get(N)
    pts = np.array(points)
    xx1 = (len(vol1)-1) * pts[:,0] - s
    yy1 = (len(vol1)-1) * pts[:,1] - s
    mask = np.sqrt(xx1**2+yy1**2) < s - 1
    x1 = xx1[mask]
    y1 = yy1[mask]
    new_vol1 = np.zeros([len(vol1),len(x1)])
    print len(x1),
    for i in xrange(len(vol1)):
        if(axis == 0):
            sub1 = circle_mask(vol1[i,:,:])
        elif(axis == 1):
            sub1 = circle_mask(vol1[:,i,:])
        else:
            sub1 = circle_mask(vol1[:,:,i])
        rot = bessel_rotate_halton(sub1, 0, x1, y1)
        new_vol1[i] = rot
    for idx, t in enumerate(thetas):
        print t, 
        new_vol2 = np.empty([len(vol2),len(x1)])
        for i in xrange(len(vol2)):
            if(axis==0):
                sub2 = circle_mask(vol2[i,:,:])
            elif(axis==1):
                sub2 = circle_mask(vol2[:,i,:])
            else:
                sub2 = circle_mask(vol2[:,:,i])
            rot = bessel_rotate_halton(sub2, t, x1, y1)
            new_vol2[i] = rot
        cost_func[idx] = cf_ssd(new_vol2,new_vol1)
    return cost_func

In [6]:
# get some test data
# head coil
head_iso_5mm = data_dict['5mm']['head']['iso'][0]
head_5deg_LR_5mm = data_dict['5mm']['head']['LR']['5deg'][0]
head_5deg_AP_5mm = data_dict['5mm']['head']['AP']['5deg'][0]
# body coil
body_iso_5mm = data_dict['5mm']['body']['iso'][0]
body_5deg_LR_5mm = data_dict['5mm']['body']['LR']['5deg'][0]
body_5deg_AP_5mm = data_dict['5mm']['body']['AP']['5deg'][0]

In [8]:
golden_section_search(bessel_cost, -10,10, 1e-2, head_iso_5mm, head_5deg_LR_5mm,0)

-2.360679775 2.360679775
2.360679775 5.27864045
5.27864045 7.08203932499
4.16407864999 5.27864045
5.27864045 5.96747752498
4.85291572496 5.27864045
4.58980337503 4.85291572496
4.85291572496 5.01552810008
4.75241575015 4.85291572496
4.69030334984 4.75241575015
4.65191577533 4.69030334984
4.69030334984 4.71402817564
4.67564060113 4.69030334984
time:  374.570827007


4.6948343883822243

In [10]:
golden_section_search(bessel_cost, 4.5,5.2, 1e-2, head_iso_5mm, head_5deg_LR_5mm,0)

4.76737620788 4.93262379212
4.66524758425 4.76737620788
4.76737620788 4.8304951685
4.72836654487 4.76737620788
4.70425724725 4.72836654487
4.72836654487 4.74326691025
time:  173.609818935


4.7478713763747793